In [84]:
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as st
# from scipy.stats import norm, chi2
# import statsmodels.api as sm

# Import Data
df = pd.read_csv("../data/13222067_cleaned.csv")
df.head()

,jenis_kelamin,usia,pendidikan,pekerjaan,penghasilan,domisili,durasi_internet,aktivitas_online_meningkat,aktivitas_online_meningkat_3bln,layanan_aktif_1,...,bayar_online_5,bayar_online_6,bayar_online_7,bayar_online_8,keluhan_online_1,keluhan_online_2,keluhan_online_3,keluhan_online_4,keluhan_online_5,keluhan_online_6
0,Pria,47,SMA,Pekerja Serabutan,Rp 2 juta – Rp 5 juta,bandung,3.0,Sama saja,NaN,Mobile Banking,...,NaN,NaN,Melalui minimarket,NaN,Barang yang diperoleh tidak sesuai dengan spes...,Barang rusak/ salah tetapi tidak dapat dikemba...,NaN,NaN,NaN,NaN
1,Wanita,19,SMA,Pelajar / Mahasiswa,< Rp 2 juta,surabaya,10.0,Ya,Keperluan mengerjakan tugas,Mobile Banking,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pria,50,S1,Karyawan Swasta,Rp 5 juta – Rp 10 juta,jakarta,6.0,Ya,Melakukan video conference,Mobile Banking,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wanita,19,SMA,Pelajar / Mahasiswa,< Rp 2 juta,bandung,5.0,Ya,Mengikuti kelas online,Mobile Banking,...,NaN,NaN,NaN,NaN,Barang yang diperoleh tidak sesuai dengan spes...,Barang rusak/ salah tetapi tidak dapat dikemba...,NaN,NaN,NaN,NaN
4,Pria,28,S1,Karyawan Swasta,Rp 5 juta – Rp 10 juta,bandung,9.0,Sama saja,NaN,Mobile Banking,...,Transfer via ATM,NaN,NaN,NaN,NaN,NaN,Pembayaran sudah dilakukan; barang tidak tersedia,NaN,Pembayaran telah dilakukan tetapi tidak terdet...,NaN


# Temp

In [85]:

def countToSeries(df, column, normalize=True, threshold=0, otherLabel="lain"):
    value_count = df[column].value_counts(normalize=normalize)
    total = value_count.sum()
    newSeries = {}
    for idx, val in value_count.items():
        if(val/total < threshold/100):
            if(otherLabel in newSeries):
                newSeries[otherLabel] += val
            else:
                newSeries[otherLabel] = 0
        else:
            newSeries[idx] = val
    series1 = pd.Series(newSeries)
    return series1

def multipleChoiceToSeries(df, column_list, normalize=True):
    value_counts = {}
    total = df[column_list[0]].shape[0]
    for col in column_list:
        value_count = df[col].value_counts(dropna=True) 
        key = value_count.keys().tolist()[0]
        val = value_count.tolist()[0]
        if(normalize):
            value_counts[key] = val/total
        else:
            value_counts[key] = val
    value_counts_series = pd.Series(value_counts)
    return value_counts_series

# Analysis + Visual Tools

In [121]:

# Confidence Interval One Sample
def ci_p(phat, n, alpha=0.05):
    z_halfAlpha = st.norm.ppf(1-(alpha/2))
    marginErr = z_halfAlpha((phat*(1-phat)/n)**0.5)
    lb = phat-marginErr
    ub = phat+marginErr
    if(lb<phat and phat<ub):
        conclusion = "Fail to Reject H0"
    else:
        conclusion = "Reject H0"

    ans = "Z-Value: {}\nConfidence Interval: {} < p < {}\nConclusion: {}".format(z_halfAlpha, lb, ub, conclusion)
    return ans

# Confidence Interval One Sample (Series)
def ci_p_series(phat_series, n, alpha=0.05):
    z_halfAlpha = st.norm.ppf(1-(alpha/2))
    lbDict = {}
    ubDict = {}
    for key, phat in phat_series.items():
        marginErr = z_halfAlpha*((phat*(1-phat)/n)**0.5)
        lbDict[key] = phat-marginErr
        ubDict[key] = phat+marginErr
    return (z_halfAlpha, pd.concat([phat_series.rename("Proportion"), pd.Series(lbDict, name="Lower"), pd.Series(ubDict, name="Upper")], axis=1).reset_index())

# Chi Square Goodness of Fit
def chiSqGoF(series, alpha=0.05): # Basically.. Is this series uniform???
    # Calculating
    n = len(series)
    nsum = series.sum()
    Ei = 1/len(series)*nsum
    chisq = 0
    for key, Oi in series.items():
        oiei2ei = ((Oi-Ei)**2 / Ei)
        chisq += oiei2ei
    chi2_alpha = st.chi2.ppf(1-alpha, n-1)
    pval = (1 - st.chi2.cdf(chisq, n-1))
    
    # Conclusion
    if(chisq > chi2_alpha):
        conclusion = "Fail to Reject H0"
    else:
        conclusion = "Reject H0"
    ans = "P-Value: {}\nchi2: {}\n Critical Region: chi2 > {}\nConclusion: {}".format(pval, chisq, chi2_alpha, conclusion)

    return ans

# Confidence Interval On Sample
def ci_p2(x1, x2, n1, n2, alpha=0.05):
    phat1 = x1/n1
    phat2 = x2/n2
    phat = (x1+x2)/(n1+n2)
    z_halfAlpha = st.norm.ppf(1-(alpha/2))
    z = (phat1-phat2)/(phat*(1-phat)*(1/n1 + 1/n2))**0.5

    lb = -z_halfAlpha
    ub = z_halfAlpha
    if(lb<z and z<ub):
        conclusion = "Fail to Reject H0"
    else:
        conclusion = "Reject H0"

    ans = "Z-Value: {}\nConfidence Interval: {} < p1-p2 < {}\nConclusion: {}".format(z ,lb, ub, conclusion)
    return ans

# Chi Square Independence
def chiSqTest(df_cont, alpha=0.05): # Basically, Is the value of A affects B?
    Ei = df_cont.copy()
    for s in df_cont.index:
        for o in df_cont.columns:
            Ei.loc[s,o] = df_cont.loc[s].sum() * df_cont.loc[:,o].sum() / df_cont.values.sum()
        
    row, col = df_cont.shape
    df = (row-1)*(col-1)
    chi2 = ((df_cont - Ei)**2/Ei).sum().sum()
    chi2_alpha = st.chi2.ppf(1-alpha, df)
    pval = 1 - st.chi2.cdf(chi2, df)

    # Conclusion
    if(chi2 > chi2_alpha):
        conclusion = "Fail to Reject H0"
    else:
        conclusion = "Reject H0"
    ans = "P-Value: {}\nchi2: {}\n Critical Region: chi2 > {}\nConclusion: {}".format(pval, chi2, chi2_alpha, conclusion)
    return ans

In [106]:

def create_plot_valuecount(filepath, df, column, title="", xlabel="", ylabel="", angle=0, threshold=0, otherLabel="lainnya", kind="bar", order=[], orientation="y"):
    # Create Series
    value_count = df[column].value_counts(normalize=True)
    total = value_count.sum()
    newSeries = {}
    for idx, val in value_count.items():
        if(val/total < threshold/100):
            if(otherLabel in newSeries):
                newSeries[otherLabel] += val
            else:
                newSeries[otherLabel] = 0
        else:
            newSeries[idx] = val
    series1 = pd.Series(newSeries)

    # Graphing
    if(order==[]):
        if(orientation=="y"):
            plt = sns.barplot(x=series1.index, y=series1.values, hue=series1.values)
        else:
            plt = sns.barplot(y=series1.index, x=series1.values, hue=series1.values)
    else:
        if(orientation=="y"):
            plt = sns.barplot(x=series1.index, y=series1.values, order=order, hue=series1.values)
        else:
            plt = sns.barplot(y=series1.index, x=series1.values, order=order, hue=series1.values)
    for p in plt.patches:
        if(p.get_height()==0):
            continue
        if(orientation=="y"):
            plt.annotate('{0:.2f}%'.format(p.get_height()*100), (p.get_x(), p.get_height()+0.01))
        else:
            plt.annotate('{0:.2f}%'.format(p.get_width()*100), (p.get_width() + 0.01, p.get_y() + p.get_height()))
    plt.legend([],[], frameon=False)
    plt.set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.tick_params(rotation=angle)

    # Analyzing
    # I forgot that the series cant be proportion series, sooo i'm gonna make another series
    z, df_ci = ci_p_series(series1, df.shape[0])
    print("---------------- Confidence Interval "+title+" ----------------")
    print("z = ", z)
    print(df_ci)
    print("----------------------------------------------------------------")
    for i in range(len(df_ci)):
        if(orientation == "y"):
            plt.plot([i,i], [df_ci["Lower"][i], df_ci["Upper"][i]], color="#74aad4")
        else:
            plt.plot([df_ci["Lower"][i], df_ci["Upper"][i]], [i,i], color="#74aad4")
    plt.figure.savefig(filepath, bbox_inches="tight")
    plt.figure.clf()

    # Create Series
    value_count = df[column].value_counts()
    total = value_count.sum()
    newSeries = {}
    for idx, val in value_count.items():
        if(val/total < threshold/100):
            if(otherLabel in newSeries):
                newSeries[otherLabel] += val
            else:
                newSeries[otherLabel] = 0
        else:
            newSeries[idx] = val
    series2 = pd.Series(newSeries)
    print("===== Chi-Square Goodness of Fit "+title+" =====")
    print(chiSqGoF(series2, alpha=0.05))
    print("===============================================")



def create_plot_multichoice(filepath, df, column_list, title="", xlabel="", ylabel="", angle=0, orientation="y"):
    # Creating series
    value_counts = {}
    total = df[column_list[0]].shape[0]
    for col in column_list:
        value_count = df[col].value_counts(dropna=True) 
        key = value_count.keys().tolist()[0]
        val = value_count.tolist()[0]
        value_counts[key] = val/total
    value_counts_series = pd.Series(value_counts)

    # Graphing
    if(orientation=="y"):
        plt = sns.barplot(x=value_counts_series.index, y=value_counts_series.values, hue=value_counts_series.values)
    else:
        plt = sns.barplot(y=value_counts_series.index, x=value_counts_series.values, hue=value_counts_series.values)
    for p in plt.patches:
        if(p.get_height()==0):
            continue
        if(orientation=="y"):
            plt.annotate('{0:.2f}%'.format(p.get_height()*100), (p.get_x(), p.get_height()+0.01))
        else:
            plt.annotate('{0:.2f}%'.format(p.get_width()*100), (p.get_width() + 0.01, p.get_y() + p.get_height()))
    plt.legend([],[], frameon=False)
    plt.set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.tick_params(rotation=angle)

    # Analyzing
    z, df_ci = ci_p_series(value_counts_series, df.shape[0])
    print("---------------- Confidence Interval "+title+" ----------------")
    print("z = ", z)
    print(df_ci)
    print("----------------------------------------------------------------")
    for i in range(len(df_ci)):
        if(orientation == "y"):
            plt.plot([i,i], [df_ci["Lower"][i], df_ci["Upper"][i]], color="#74aad4")
        else:
            plt.plot([df_ci["Lower"][i], df_ci["Upper"][i]], [i,i], color="#74aad4")
    plt.figure.savefig(filepath, bbox_inches="tight")
    plt.figure.clf()

    value_counts = {}
    total = df[column_list[0]].shape[0]
    for col in column_list:
        value_count = df[col].value_counts(dropna=True) 
        key = value_count.keys().tolist()[0]
        val = value_count.tolist()[0]
        value_counts[key] = val
    value_counts_series2 = pd.Series(value_counts)
    print("===== Chi-Square Goodness of Fit "+title+" =====")
    print(chiSqGoF(value_counts_series2, alpha=0.05))
    print("===============================================")


# 1 Variable

In [107]:
# A. Demografi
demografi = ["jenis_kelamin", "usia", "pendidikan", "pekerjaan", "penghasilan", "domisili", "durasi_internet", "aktivitas_online_meningkat", "aktivitas_online_meningkat_3bln"]
demografi_title = ["Jenis Kelamin", "Usia", "Pendidikan Terakhir", "Pekerjaan", "Penghasilan", "Domisili", "Durasi Pemakaian Internet", "Aktivitas Online Terakhir", "Aktivitas Online Meningkat 3 Bulan Terakhir"]

create_plot_valuecount("../output2/demografi/plot_jenis_kelamin", df, "jenis_kelamin", title="Jenis Kelamin", ylabel="%")
create_plot_valuecount("../output2/demografi/plot_pendidikan", df, "pendidikan", title="Pendidikan", ylabel="%", order=["SD", "SMP", "SMK", "SMA", "D1", "D2", "D3", "S1", "S2", "S3"])
create_plot_valuecount("../output2/demografi/plot_pekerjaan", df, "pekerjaan", title="Pekerjaan", ylabel="%", threshold=3, otherLabel="<3%", angle=30)
create_plot_valuecount("../output2/demografi/plot_penghasilan", df, "penghasilan", title="Penghasilan", ylabel="%", angle=45, order=['< Rp 2 juta', 'Rp 2 juta – Rp 5 juta', 'Rp 5 juta – Rp 10 juta', '> Rp 10 juta'])
create_plot_valuecount("../output2/demografi/plot_domisili", df, "domisili", title="Domisili", ylabel="%", threshold=3, otherLabel="<3%", angle=45)

---------------- Confidence Interval Jenis Kelamin ----------------
z =  1.959963984540054
    index  Proportion     Lower     Upper
0    Pria    0.510417  0.452683  0.568150
1  Wanita    0.489583  0.431850  0.547317
----------------------------------------------------------------
===== Chi-Square Goodness of Fit Jenis Kelamin =====
P-Value: 0.7236736098317629
chi2: 0.125
 Critical Region: chi2 > 3.841458820694124
Conclusion: Reject H0
---------------- Confidence Interval Pendidikan ----------------
z =  1.959963984540054
  index  Proportion     Lower     Upper
0   SMA    0.458333  0.400788  0.515878
1    S1    0.375000  0.319088  0.430912
2    S2    0.086806  0.054289  0.119322
3    D3    0.055556  0.029101  0.082010
4    D4    0.006944 -0.002646  0.016535
5    D1    0.003472 -0.003321  0.010266
6   SMK    0.003472 -0.003321  0.010266
7   SMP    0.003472 -0.003321  0.010266
8    SD    0.003472 -0.003321  0.010266
9    S3    0.003472 -0.003321  0.010266
--------------------------------

<Figure size 640x480 with 0 Axes>

In [108]:
# B. Bank
layanan_aktif_cols = ["layanan_aktif_1", "layanan_aktif_2", "layanan_aktif_3", "layanan_aktif_4", "layanan_aktif_5", "layanan_aktif_6"]
bank_cols = ["bank_1", "bank_2", "bank_3", "bank_4", "bank_4", "bank_6", "bank_7"]
freq_bank_cols = ["freq_atm", "freq_internet_bank", "freq_mobile_bank", "freq_sms_bank", "freq_cabang"]
fitur_disukai_cols = ["fitur_cekSaldo", "fitur_mutasi", "fitur_transfer_sesamabank", "fitur_transfer_antarbank", "fitur_pulsa", "fitur_tagihan"]
keluhan_cols = ["keluhan_1", "keluhan_2", "keluhan3", "keluhan 4"]

create_plot_multichoice("../output2/bank/plot_layanan_aktif", df, layanan_aktif_cols, title="Layanan Aktif", ylabel="Jumlah", angle=30)
create_plot_multichoice("../output2/bank/plot_bank", df, bank_cols, title="Bank", ylabel="Jumlah", angle=30)
create_plot_multichoice("../output2/bank/plot_keluhan_bank", df, keluhan_cols, title="Keluhan Bank", xlabel="Jumlah", orientation="x")

---------------- Confidence Interval Layanan Aktif ----------------
z =  1.959963984540054
                     index  Proportion     Lower     Upper
0           Mobile Banking    0.836806  0.794126  0.879485
1                      ATM    0.586806  0.529936  0.643675
2             Kartu Kredit    0.090278  0.057180  0.123375
3         Internet Banking    0.204861  0.158249  0.251474
4  Datang ke Kantor Cabang    0.090278  0.057180  0.123375
5              SMS Banking    0.038194  0.016059  0.060330
----------------------------------------------------------------
===== Chi-Square Goodness of Fit Layanan Aktif =====
P-Value: 0.0
chi2: 501.0375939849625
 Critical Region: chi2 > 11.070497693516351
Conclusion: Fail to Reject H0
---------------- Confidence Interval Bank ----------------
z =  1.959963984540054
             index  Proportion     Lower     Upper
0         Bank BNI    0.222222  0.174208  0.270237
1     Bank Mandiri    0.274306  0.222777  0.325834
2         Bank BCA    0.569444  

<Figure size 640x480 with 0 Axes>

In [109]:
# C. Emoney
emoney_pilihan_cols = ["emoney_pilihan1", "emoney_pilihan2", "emoney_pilihan3", "emoney_pilihan4", "emoney_pilihan5", "emoney_pilihan6"
        , "emoney_pilihan7", "emoney_pilihan8", "emoney_pilihan9", "emoney_pilihan10", "emoney_pilihan11"]
alasan_emoney_cols = ["alasan_emoney_1", "alasan_emoney_2", "alasan_emoney_3", "alasan_emoney_4", "alasan_money_5"]
penggunaan_emoney_cols = ["penggunaan_emoney_1", "penggunaan_emoney_2", "penggunaan_emoney_3", "penggunaan_emoney_4", "penggunaan_emoney_5"
        , "penggunaan_emoney_6", "penggunaan_emoney_7", "penggunaan_emoney_8", "penggunaan_emoney_9", "penggunaan_emoney_10"]
cara_topup_cols = ["cara_topup_emoney_1", "cara_topup_emoney_2", "cara_topup_emoney_3", "cara_topup_emoney_4", "cara_topup_emoney_5", "cara_topup_emoney_6", "cara_topup_emoney_7"]
keluhan_emoney_cols = ["keluhan_emoney_1", "keluhan_emoney_2", "keluhan_emoney_3", "keluhan_emoney_4"]
penggunaan_qr_cols = ["penggunaan_qr_1", "penggunaan_qr_2", "penggunaan_qr_3", "penggunaan_qr_4"]
lokasi_qr_cols = ["lokasi_qr_1", "lokasi_qr_2", "lokasi_qr_3", "lokasi_qr_4", "lokasi_qr_5", "lokasi_qr_6"]

create_plot_multichoice("../output2/emoney/plot_emoney", df, emoney_pilihan_cols, title="E-money Pilihan", ylabel="jumlah", angle=90)
create_plot_multichoice("../output2/emoney/plot_alasan_emoney", df, alasan_emoney_cols, title="Alasan Memilih E-money", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/emoney/plot_penggunaan_emoney", df, penggunaan_emoney_cols, title="Penggunaan E-Money dalam Kehidupan Sehari-hari", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/emoney/plot_cara_topup", df, cara_topup_cols, title="Cara Top-Up E-Money", xlabel="jumlah", orientation="x")

create_plot_valuecount("../output2/emoney/plot_nominal_topup", df, "nominal_topup", title="Nominal Top-Up", ylabel="jumlah", orientation="x")
create_plot_valuecount("../output2/emoney/plot_frekuensi_topup", df, "freq_topup", title="Frekuensi Top-Up", ylabel="jumlah", orientation="x")

create_plot_multichoice("../output2/emoney/plot_keluhan_emoney", df, keluhan_emoney_cols, title="Keluhan dalam Menggunakan E-Money", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/emoney/plot_penggunaan_qr", df, penggunaan_qr_cols, title="Penggunaan QR dalam Kehidupan Sehari-hari", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/emoney/plot_lokasi_qr", df, lokasi_qr_cols, title="Lokasi Penggunaan QR", xlabel="jumlah", orientation="x")

---------------- Confidence Interval E-money Pilihan ----------------
z =  1.959963984540054
                       index  Proportion     Lower     Upper
0                      GoPay    0.750000  0.699991  0.800009
1                  ShopeePay    0.597222  0.540578  0.653866
2                        OVO    0.614583  0.558374  0.670793
3                       Dana    0.402778  0.346134  0.459422
4             eMoney Mandiri    0.298611  0.245756  0.351466
5                  Flazz BCA    0.222222  0.174208  0.270237
6                    LinkAja    0.135417  0.095899  0.174934
7   Tidak Menggunakan eMoney    0.059028  0.031809  0.086247
8                     Brizzi    0.069444  0.040085  0.098803
9                    JakCard    0.010417 -0.001309  0.022142
10                     iSaku    0.003472 -0.003321  0.010266
----------------------------------------------------------------
===== Chi-Square Goodness of Fit E-money Pilihan =====
P-Value: 0.0
chi2: 715.9012074643248
 Critical Region: 

<Figure size 640x480 with 0 Axes>

In [110]:
belanja_online_meningkat_cols = ["belanja_online_1", "belanja_online_2", "belanja_online_3", "belanja_online_4", "belanja_online_5"
        , "belanja_online_6", "belanja_online_7", "belanja_online_8", "belanja_online_9", "belanja_online_10"]
belanja_barang_cols = ["belanja_fashion", "belanja_groceries", "belanja_hp", "belanja_laptop", "belanja_elektronik"
        , "belanja_kosmetik", "belanja_hobi", "belanja_pulsa", "belanja_makanan", "belanja_pesawat", "belanja_bioskop"]
tempat_belanja_online_cols = ["tempat_belanja_online_1", "tempat_belanja_online_2", "tempat_belanja_online_3", "tempat_belanja_online_4"
        , "tempat_belanja_online_5", "tempat_belanja_online_6", "tempat_belanja_online_7", "tempat_belanja_online_8"]
ecommerce_cols = ["ecommerce_1", "ecommerce_2", "ecommerce_3", "ecommerce_4", "ecommerce_5", "ecommerce_6"
        , "ecommerce_7", "ecommerce_8", "ecommerce_9", "ecommerce_10", "ecommerce_11", "ecommerce_12"]
alasan_belanja_offline_cols = ["alasan_offline_1", "alasan_offline_2", "alasan_offline_3", "alasan_offline_4", "alasan_offline_5", "alasan_offline_6", "alasan_offline_7"]
cara_bayar_online_cols = ["bayar_online_1", "bayar_online_2", "bayar_online_3", "bayar_online_4", "bayar_online_5", "bayar_online_6", "bayar_online_7", "bayar_online_8"]
keluhan_online_cols = ["keluhan_online_1", "keluhan_online_2", "keluhan_online_3", "keluhan_online_4", "keluhan_online_5", "keluhan_online_6"]

create_plot_valuecount("../output2/belanja/plot_freq_belanja_6bln", df, "freq_belanja_6bln", title="Frekuensi Belanja 6 Bulan Terakhir", xlabel="jumlah", orientation="x")

create_plot_multichoice("../output2/belanja/plot_belanja_online_meningkat", df, belanja_online_meningkat_cols, title="Belanja Online Meningkat 6 Bulan Terakhir", xlabel="jumlah", orientation="x")
# create_plot_multichoice("../output/belanja/plot_belanja_online_barang", df, belanja_barang_cols, title="Kecenderungan Barang Belanja Online", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/belanja/plot_tempat_belanja_online", df, tempat_belanja_online_cols, title="Tempat Belanja Online", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/belanja/plot_ecommerce", df, ecommerce_cols, title="Ecommerce Pilihan", xlabel="x", angle=90)

create_plot_valuecount("../output2/belanja/plot_food_delivery", df, "food_delivery", title="Layanan Food Delivery yang Sering Digunakan", ylabel="jumlah", orientation="y", angle=70)

create_plot_multichoice("../output2/belanja/plot_alasan_belanja_offline", df, alasan_belanja_offline_cols, title="Alasan Belanja Offline", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/belanja/plot_cara_bayar_online", df, cara_bayar_online_cols, title="Cara Bayar Online", xlabel="jumlah", orientation="x")
create_plot_multichoice("../output2/belanja/plot_keluhan_online", df, keluhan_online_cols, title="Keluhan Belanja Online", xlabel="jumlah", orientation="x")

---------------- Confidence Interval Frekuensi Belanja 6 Bulan Terakhir ----------------
z =  1.959963984540054
       index  Proportion     Lower     Upper
0      Tetap    0.414035  0.357149  0.470921
1  Meningkat    0.361404  0.305920  0.416887
2    Menurun    0.224561  0.176367  0.272755
----------------------------------------------------------------
===== Chi-Square Goodness of Fit Frekuensi Belanja 6 Bulan Terakhir =====
P-Value: 0.0002804970448303523
chi2: 16.357894736842105
 Critical Region: chi2 > 5.991464547107979
Conclusion: Fail to Reject H0
---------------- Confidence Interval Belanja Online Meningkat 6 Bulan Terakhir ----------------
z =  1.959963984540054
                                               index  Proportion     Lower  \
0          Membeli makanan dan minuman secara online    0.315972  0.262280   
1               Belanja fashion & mode secara online    0.250000  0.199991   
2                      Pembelian pulsa secara online    0.225694  0.177414   
3  Belanj

<Figure size 640x480 with 0 Axes>

# 2 variable

In [133]:
series_test = multipleChoiceToSeries(df, emoney_pilihan_cols, normalize=False)
print(series_test)
# print(series_test["GoPay"])

print(ci_p2(series_test["GoPay"], series_test["ShopeePay"], df.shape[0], df.shape[0]))
print(ci_p2(series_test["GoPay"], series_test["OVO"], df.shape[0], df.shape[0]))
print(ci_p2(series_test["ShopeePay"], series_test["OVO"], df.shape[0], df.shape[0]))

GoPay                       216
ShopeePay                   172
OVO                         177
Dana                        116
eMoney Mandiri               86
Flazz BCA                    64
LinkAja                      39
Tidak Menggunakan eMoney     17
Brizzi                       20
JakCard                       3
iSaku                         1
dtype: int64
Z-Value: 1.959963984540054
Confidence Interval: -1.959963984540054 < p1-p2 < 1.959963984540054
Conclusion: Fail to Reject H0
Z-Value: 1.959963984540054
Confidence Interval: -1.959963984540054 < p1-p2 < 1.959963984540054
Conclusion: Fail to Reject H0
Z-Value: 1.959963984540054
Confidence Interval: -1.959963984540054 < p1-p2 < 1.959963984540054
Conclusion: Fail to Reject H0
